<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Run GriddedCCP

In [1]:
cd ..

/private/home/yannd/projects/Neural-Process-Family


In [2]:
import neuralproc

In [3]:
import submitit
from train_imgs import main, parse_arguments

log_folder="logs/%j"

In [4]:
def repeat_list(l, n):
    return [i for i in l for _ in range(n)]

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train=True 
        return submitit.utils.DelayedSubmission(self, args)
    
    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)

In [26]:
datasets_32 = ["mnist","celeba32",  "svhn"]
datasets_64 = ["celeba64",  "zs-multi-mnist"]

Grided_ccp_kwargs = [("GridedCCP", ""),
                   ("GridedCCP_rbf", "--is-rbf"),
                   ("GridedCCP_large", "--n-blocks 7 --kernel-size 11 --init-kernel-size 11"),
                   ("GridedCCP_no_density", "--is-no-density"),
                   ("GridedCCP_no_norm", "--is-no-normalization")]

args_32_ccp = [parse_arguments("GridedCCP {} --starting-run {} --name {} --chckpnt-dirname results/iclr/ {}".format(d, s, name, other).split())
        for s in range(2)
        for d in datasets_32
        for name,other in Grided_ccp_kwargs]

args_32_anp = [parse_arguments("SelfAttnCNP {} --starting-run {} --chckpnt-dirname results/iclr/ ".format(d, s).split())
                for s in range(6)
                for d in datasets_32]

args_64_ccp = [parse_arguments("GridedCCP {} --starting-run {} --name {} --chckpnt-dirname results/iclr/ {}".format(d, s, name, other).split())
        for s in range(3)
        for d in datasets_64
        for name,other in Grided_ccp_kwargs]

args_64_anp = [parse_arguments("SelfAttnCNP {} --starting-run {} --batch-size 6 --chckpnt-dirname results/iclr/ ".format(d, s).split())
                for s in range(6)
                for d in ["zs-multi-mnist"]]

In [27]:
executor_32_ccp=submitit.SlurmExecutor(folder=log_folder, max_num_timeout=3)
executor_32_ccp.update_parameters(num_gpus=1, 
                                   time=60*24*2,  
                                   cpus_per_task=10, 
                                   mem='32GB',
                                   ntasks_per_node=3
                                  )

executor_32_anp=submitit.SlurmExecutor(folder=log_folder, max_num_timeout=3)
executor_32_anp.update_parameters(num_gpus=1, 
                                   time=60*24*2,  
                                   cpus_per_task=10, 
                                   mem='32GB',
                                  )

executor_64_ccp=submitit.SlurmExecutor(folder=log_folder, max_num_timeout=3)
executor_64_ccp.update_parameters(num_gpus=1, 
                                   time=60*24*2,  
                                   cpus_per_task=10, 
                                   mem='32GB',
                                   ntasks_per_node=2
                                  )

executor_64_anp=submitit.SlurmExecutor(folder=log_folder, max_num_timeout=3)
executor_64_anp.update_parameters(num_gpus=1, 
                                   time=60*24*3,  
                                   cpus_per_task=10, 
                                   mem='32GB',
                                  constraint="volta32gb"
                                  )

In [8]:
job_32_ccp = executor_32_ccp.map_array(Run(), args_32_ccp)
job_32_anp = executor_32_anp.map_array(Run(), args_32_anp)
job_64_ccp = executor_64_ccp.map_array(Run(), args_64_ccp)


In [28]:
job_64_anp = executor_64_anp.map_array(Run(), args_64_anp)

In [16]:
############################################################

In [72]:
job_32_ccp

[SlurmJob<job_id=17816286_0, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_1, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_2, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_3, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_4, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_5, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_6, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_7, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_8, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_9, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_10, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_11, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_12, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_13, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_14, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816286_15, task_id=None, state=

In [73]:
job_32_anp

[SlurmJob<job_id=17816289_0, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_1, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_2, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_3, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_4, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_5, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_6, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_7, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_8, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_9, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_10, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_11, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_12, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_13, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_14, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_15, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17816289_16, task

In [74]:
job_64_ccp

[SlurmJob<job_id=17816290_0, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_1, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_2, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_3, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_4, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_5, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_6, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_7, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_8, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_9, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_10, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_11, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_12, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_13, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_14, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=17816290_15, task_id=None, state=

In [75]:
job_64_anp

[SlurmJob<job_id=17818813_0, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17818813_1, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17818813_2, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17818813_3, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17818813_4, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17818813_5, task_id=0, state="COMPLETED">]

In [66]:
for j in job_64_anp:
    print("--------------------------------")
    print(j.stdout())

--------------------------------
submitit INFO (2019-09-18 11:04:33,803) - Starting with JobEnvironment(job_id=17818813_0, hostname=learnfair0331, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2019-09-18 11:04:33,803) - Loading pickle: /private/home/yannd/projects/Neural-Process-Family/logs/17818813_0/17818813_0_submitted.pkl

--- Training zs-multi-mnist/SelfAttnCNP/run_0 ---

  epoch    train_loss    valid_loss    cp        dur
-------  ------------  ------------  ----  ---------
      1       -1.2168       -1.2257     +  2817.8178
      2       -1.2769       -1.2381     +  2651.9996
      3       -1.2915       -1.2604     +  2797.2357
      4       -1.2982       -1.2528        2808.5440
      5       -1.3032       -1.2538        2747.8785
      6       -1.3067       -1.2554        2815.3366
      7       -1.3096       -1.2347        2843.8658
      8       -1.3117       -1.2320        2826.9894
      9       -1.3139       -1.2520        2637.1249
     10       -1.3155 

In [46]:
for j in job_64_anp:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
/private/home/yannd/projects/Neural-Process-Family/neuralproc/predefined/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/private/home/yannd/projects/Neural-Process-Family/neuralproc/predefined/mlp.py:67: UserWarning: hidden_size=32 smaller than output=256 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))

--------------------------------
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
/private/home/yannd/projects/Neural-Process-Family/neuralproc/predefined/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, out

In [13]:
for j in job:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
submitit WARNING (2019-09-18 06:21:13,329) - Caught USR1 signal 10 on learnfair0223: this job is timed-out.
INFO:submitit:Calling checkpoint method.
slurmstepd-learnfair0223: error: *** STEP 17806078.0 ON learnfair0223 CANCELLED AT 2019-09-18T06:21:13 DUE TO JOB REQUEUE ***
slurmstepd-learnfair0223: error: *** JOB 17806078 ON learnfair0223 CANCELLED AT 2019-09-18T06:21:13 DUE TO JOB REQUEUE ***
INFO:submitit:Requeued job 17806077_0 (2 remaining timeouts)
INFO:submitit:Exiting gracefully after preemption/timeout.
submitit WARNING (2019-09-18 06:21:13,585) - Bypassing sigterm.

--------------------------------
submitit WARNING (2019-09-18 06:21:13,330) - Caught USR1 signal 10 on learnfair0223: this job is timed-out.
INFO:submitit:Calling checkpoint method.
INFO:submitit:Requeued job 17806077_1 (2 remaining timeouts)
INFO:submitit:Exiting gracefully after preemption/timeout.
slurmstepd-learnfair0223: error: *** STEP 17806079.0 ON learnfair0223 CANCELLED AT